### Script Purpose
- Concatenate text and NTEE codes.
- Prepare test dataset (`df_train.pkl.gz`).

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import os
import math
from spellchecker import SpellChecker
import nltk
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

In [2]:
df_txt=pd.DataFrame()
file_list_txt=os.listdir('../../dataset/intermediary/EIN_TXT_2014_18.pkl.gz/')
for file in file_list_txt:
    df_txt=pd.concat([df_txt,
                      pd.read_pickle('../../dataset/intermediary/EIN_TXT_2014_18.pkl.gz/'+file, compression='gzip')
                     ])
df_txt.sample(3)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR,95_and_before
1825837,NaN,621202940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
748489,9.349113e+13,412083662,EFILE,NaN,NaN,,FUNDING ASSISTANCE TO LOCAL ORGANIZATIONS THAT...,NaN,NaN,NaN,NaN,2.017413e+17,14803397.0,990PF,10/4/2017 12:57:36 PM,PROFESSIONAL MORTGAGE CORPORATION CHARITABLE F...,201612.0,2017.0,0.0
34823,9.349325e+13,431820245,EFILE,NaN,NaN,NaN,NaN,,TO PROMOTE JUNIOR AMATUER GOLF IN MISSOURI,"IN 2016, 6 INSTRUCTIONAL JUNIOR GOLF CLINICS W...",TO PROMOTE JUNIOR AMATUER GOLF IN MISSOURI,2.017025e+17,14840780.0,990,10/16/2017 1:59:44 PM,MISSOURI JUNIOR GOLF SCHOLARSHIP FOUNDATION,201612.0,2017.0,0.0


In [3]:
df_ntee=pd.read_pickle('../../dataset/intermediary/df_bmf_14_16.pkl.gz', compression='gzip')








In [4]:
list(df_ntee.columns)
df_ntee.isnull().sum()

EIN                  0
FIPS              4577
NTEECC               0
FILER                0
ZFILER               0
NAME                 0
ADDRESS             15
CITY                 0
STATE                0
ZIP5                 0
GEN                  0
SUBSECCD             0
RULEDATE             0
FNDNCD               0
TAXPER          946185
FRCD                 0
PFFRCD         4633436
ACCPER               0
ASSETS               0
INCOME               0
SEC_NAME       3345818
NTEE1                0
LEVEL1               0
LEVEL2               0
LEVEL3               0
LEVEL4               0
MAJGRPB              0
OUTNCCS              0
OUTREAS        4624418
NTEESRC        1668777
ntmaj10              0
ntmaj12              0
ntmaj5               0
nteeFinal            0
nteeFinal1           0
RandNum              0
nteeConf            88
MSA_NECH        659272
PMSA           3600920
cFiler               0
czFiler              0
cTaxPer         931376
cAssets        2563990
cTotRev    

In [5]:
df_ein_ntee=df_ntee[['EIN', 'NTEE1', 'STATE', 'CITY', 'FIPS', 'PMSA', 'MSA_NECH', 'ADDRESS']]
#Modified so we can include these other locational variable to sort
df_ein_ntee.drop_duplicates(inplace=True)

/Users/Wolfalev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df_ein_ntee.sample(3)

,EIN,NTEE1,STATE,CITY,FIPS,PMSA,MSA_NECH,ADDRESS
285518,237127195,Y,NE,CORTLAND,31067.0,NaN,0.0,PO BOX 46
1193359,611176665,X,KY,STONEY FORK,21013.0,NaN,0.0,GENERAL DELIVERY
2083409,274690768,N,CA,SAN MATEO,6081.0,7360.0,7362.0,330 W 20TH AVE


In [7]:
df_txt_ntee=df_txt.merge(df_ein_ntee, on='EIN', how='outer')
df_txt_ntee.sample(10)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR,95_and_before,NTEE1,STATE,CITY,FIPS,PMSA,MSA_NECH,ADDRESS
1172200,9.349118e+13,521608565,EFILE,NaN,NaN,,,NaN,NaN,NaN,NaN,2.015418e+17,12819513.0,990PF,8/27/2015,MEREDYTH ANNE DASBURG FOUNDATION,201412.0,2015.0,1.0,T,FL,KEY BISCAYNE,12086.0,NaN,5000.0,791 CRANDON BLVD PH 8
5044265,NaN,800953753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,MI,CLINTON TWP,26099.0,NaN,2160.0,19051 LAUREN DR
3896230,NaN,10830621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NJ,TRENTON,34021.0,8480.0,5602.0,PO BOX 5186
4952419,NaN,720805641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,LA,PEARL RIVER,22103.0,NaN,5560.0,66192 SAINT MARY DR
4758283,NaN,474243798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,IN,INDIANAPOLIS,18097.0,NaN,3480.0,1340 MEADOWBROOK DR
4851297,NaN,562637491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,CA,FOLSOM,6067.0,6920.0,6922.0,PO BOX 1508
1911033,NaN,251457120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,W,PA,CAMBRIDG SPGS,42039.0,NaN,0.0,419 MCCLELLAN ST
2014127,NaN,431345137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,N,MO,KANSAS CITY,29165.0,NaN,3760.0,10900 N POMONA AVE
3853993,9.349113e+13,46717573,EFILE,NaN,NaN,,,NaN,NaN,NaN,NaN,2.017313e+17,14805438.0,990PF,10/4/2017 7:34:24 PM,BARRY M JAYE FAMILY CHARITABLE TRUST,201612.0,2017.0,1.0,T,MA,NEEDHAM,25021.0,NaN,1123.0,C/O BRAVER WLTH MGMT 117 KENDRICK
2210961,9.349217e+13,521992384,EFILE,"TO PROVIDE WOMEN OPPORTUNITIES TO LEARN, PLAY,...","TO PROVIDE WOMEN OPPORTUNITIES TO LEARN, PLAY,...",NaN,NaN,"TO PROVIDE WOMEN OPPORTUNITIES TO LEARN, PLAY,...",NaN,NaN,NaN,2.017117e+17,14726395.0,990EO,9/12/2017 1:26:53 PM,EASTERN SHORE CHAPTER OF THE EXECUTIVE WOMENS ...,201612.0,2017.0,0.0,N,MD,OCEAN PINES,24047.0,NaN,NaN,5 NOTTINGHAM LN


In [8]:
df_txt_ntee['mission']=df_txt_ntee['IRS990_p1_ActvtyOrMssnDsc'].combine_first(df_txt_ntee['IRS990_p3_MssnDsc']).combine_first(df_txt_ntee['IRS990EZ_p3_PrmryExmptPrpsTxt'])
df_txt_ntee['prgrm_dsc']=df_txt_ntee['IRS990_p3_DscS'].fillna('')+'##'+df_txt_ntee['IRS990ScheduleO_ExplntnTxt'].fillna('')+'##'+\
                                      df_txt_ntee['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt'].fillna('')+'##'+df_txt_ntee['IRS990ScheduleO_ExplntnTxt'].fillna('')+'##'+\
                                      df_txt_ntee['IRS990PF_p9a_DscrptnTxt'].fillna('')+'##'+df_txt_ntee['IRS990PF_p16b_RltnshpSttmntTxt'].fillna('')

In [9]:
def func_clean_str(string):
    if str(string)=='nan':
        return ''
    else:
        string_set=set([s for s in string.split('##') if s!='']) # Remove duplicates.
        return '; '.join(string_set) # Join together.

In [10]:
df_txt_ntee['mission']=df_txt_ntee['mission'].map(func_clean_str)
df_txt_ntee['prgrm_dsc']=df_txt_ntee['prgrm_dsc'].map(func_clean_str)

In [11]:
df_train=df_txt_ntee[(~df_txt_ntee['mission'].duplicated())] # Drop duplicated mission descriptions.
df_train=df_train[(~df_train['prgrm_dsc'].duplicated())] # Drop duplicated program descriptions.
df_train=df_train[(~df_train['NTEE1'].isna())] # Drop NTEE==NaN.
# Change to upper case.
df_train['mission']=df_train['mission'].map(str.upper)
df_train['prgrm_dsc']=df_train['prgrm_dsc'].map(str.upper)
df_train['NTEE1']=df_train['NTEE1'].map(str.upper)
len(df_train)

449709

In [12]:
# Spell check function. Return corrected word if unknown; return original word if known.
def spellcheck(doc):
    word_string_list=nltk.word_tokenize(doc)
    return [SpellChecker().correction(word=s).upper() for s in word_string_list]

# Use multi-processing instead of df.apply, much faster.
p=Pool(48)

In [23]:
df_train_tx = df_train.loc[(df_train.STATE == 'TX')]

In [25]:
print(len(df_train_tx))
df_train_tx.groupby('NTEE1').count()

27285


,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR,95_and_before,STATE,CITY,FIPS,PMSA,MSA_NECH,ADDRESS,mission,prgrm_dsc
NTEE1,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,2147,2147,2147,947,947,0,0,2147,1200,1200,1200,2147,2147,2147,2147,2147,2147,2147,2147,2147,2147,2147,12,2146,2147,2147,2147
B,3749,3749,3749,1541,1541,0,0,3749,2208,2208,2208,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,14,3744,3749,3749,3749
C,407,407,407,156,156,0,0,407,251,251,251,407,407,407,407,407,407,407,407,407,407,407,3,407,407,407,407
D,635,635,635,325,325,0,0,635,310,310,310,635,635,635,635,635,635,635,635,635,635,635,2,634,635,635,635
E,1493,1493,1493,315,315,0,0,1493,1178,1178,1178,1493,1493,1493,1493,1493,1493,1493,1493,1493,1493,1493,10,1491,1493,1493,1493
F,495,495,495,152,152,0,0,495,343,343,343,495,495,495,495,495,495,495,495,495,495,495,0,493,495,495,495
G,746,746,746,320,320,0,0,746,426,426,426,746,746,746,746,746,746,746,746,746,746,746,5,746,746,746,746
H,135,135,135,45,45,0,0,135,90,90,90,135,135,135,135,135,135,135,135,135,135,135,1,135,135,135,135
I,580,580,580,238,238,0,0,580,342,342,342,580,580,580,580,580,580,580,580,580,580,580,2,580,580,580,580


407

In [ ]:
# Correct mission.
df_train_tx['mission_spellchk']=p.map(spellcheck, df_train['mission'])

In [1]:
# Correct prgrm dsc.
df_train_tx['prgrm_dsc_spellchk']=p.map(spellcheck, df_train['prgrm_dsc'])

NameError: name 'p' is not defined

**Multi-processing vs. Dataframe.apply.**
```Python
>>> from time import time
>>> p=Pool(48)
>>> t1=time()
>>> t=p.map(spellcheck, df_train['mission'][0:20])
>>> print((time()-t1)/60)
0.20710660219192506

>>> from time import time
>>> t1=time()
>>> t=df_train['mission'][0:20].apply(spellcheck)
>>> print((time()-t1)/60)
0.9466491937637329
```

In [28]:
list(df_train_tx.columns)

['DLN',
 'EIN',
 'FILING_TYPE',
 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt',
 'IRS990EZ_p3_PrmryExmptPrpsTxt',
 'IRS990PF_p16b_RltnshpSttmntTxt',
 'IRS990PF_p9a_DscrptnTxt',
 'IRS990ScheduleO_ExplntnTxt',
 'IRS990_p1_ActvtyOrMssnDsc',
 'IRS990_p3_DscS',
 'IRS990_p3_MssnDsc',
 'OBJECT_ID',
 'RETURN_ID',
 'RETURN_TYPE',
 'SUB_DATE',
 'TAXPAYER_NAME',
 'TAX_PERIOD',
 'YEAR',
 '95_and_before',
 'NTEE1',
 'mission',
 'prgrm_dsc']

In [23]:
df_to_write=df_train_tx
num_file=10
file_path_name='../../dataset/df_train_enviro.pkl.gz/df_train_enviro.pkl.gz'
for index in range(0, df_to_write.iloc[-1].name+1, math.ceil(df_to_write.iloc[-1].name/num_file)):
    df_temp=df_to_write.loc[index:index+math.ceil(df_to_write.iloc[-1].name/num_file)-1]
    df_temp.to_pickle(file_path_name+'-'+str(df_temp.iloc[0].name)+'-'+str(df_temp.iloc[-1].name), compression='gzip')

In [144]:
# Test reading file.
import os
file_list=os.listdir('../../dataset/df_train_enviro.pkl.gz/')
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, 
                       pd.read_pickle('../../dataset/df_train_enviro.pkl.gz/'+file, compression='gzip')])
len(df_test)

229472

In [25]:
# Test reading file.
import os
file_list=os.listdir('../../dataset/df_train_enviro.pkl.gz/')
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, 
                       pd.read_pickle('../../dataset/df_train_enviro.pkl.gz/'+file, compression='gzip')])
len(df_test)

234027